#### Challenge 50: Hashing with CBC-MAC

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
import cryptopals as cp

<div class="alert alert-block alert-info">   

Hashing with CBC-MAC

Sometimes people try to use CBC-MAC as a hash function.

This is a bad idea. Matt Green explains:

> To make a long story short: cryptographic hash functions are public functions (i.e., no secret key) that have the property of collision-resistance (it's hard to find two messages with the same hash). MACs are keyed functions that (typically) provide message unforgeability -- a very different property. Moreover, they guarantee this only when the key is secret. 

Let's try a simple exercise.

Hash functions are often used for code verification. This snippet of JavaScript (with newline):
    
```
alert('MZA who was that?');
```   
</br>    

Hashes to `296b8d7cb78a243dda4d0a61d33bbdd1` under CBC-MAC with a key of `YELLOW SUBMARINE` and a `0` IV.

Forge a valid snippet of JavaScript that alerts `Ayo, the Wu is back!` and hashes to the same value. Ensure that it runs in a browser.
    
<div class="alert alert-block alert-warning">  
    
#### **Extra Credit**

Write JavaScript code that downloads your file, checks its CBC-MAC, and inserts it into the DOM iff it matches the expected hash.

</div>
</div>

---


In [37]:
key = b'YELLOW SUBMARINE'
IV = b'\x00' * 16
CORRECT_MAC = '296b8d7cb78a243dda4d0a61d33bbdd1'


original_js_code  = cp.PKCS7_pad(b"alert('MZA who was that?');\n")

# Check that our CBC_MAC matches expected value for the original javascript snippet
MAC = cp.CBC_MAC(original_js_code, key, IV, False).hex()
print(MAC)
assert(MAC == CORRECT_MAC)

296b8d7cb78a243dda4d0a61d33bbdd1


That worked.  So we're doing the CBC-MAC the same way as the folks that made this challenge...i.e., using AES-128 as the encryption function for CBC-MAC.

Based on the quote, I'm assuming it's ok to assume the key is public in this case.  

To get this to work, we have to forge a piece of code that will provide the last block of AES with an input that matches the last block of input to the AES-CBC-MAC.  Unlike hash compression functions, AES is reversible...

In [52]:
last_AES_input = cp.AESDecrypt(bytes.fromhex(CORRECT_MAC), key)

# I'm going to assume I can add random garbage after a comment indicator..start with this:
malicious_js_code = cp.PKCS7_pad(b"alert('Ayo, the Wu is back!'); //")

# Calculate MAC of *my* code
malicious_MAC = cp.CBC_MAC(malicious_js_code, key, IV, False)

# Now, take this as the input to an additional round of the CBC-MAC and force it to match input to last round of the valid code..
evil_block = cp.bitwise_xor(malicious_MAC, last_AES_input)
malicious_js_code = malicious_js_code + evil_block

# Check to see if the manipulated code generates the same MAC as the original authentic code:

malicious_MAC_final = cp.CBC_MAC(malicious_js_code, key, IV, False)
print('Malicous Javascript:\n\n', malicious_js_code, '\n')
assert(malicious_MAC_final.hex() == CORRECT_MAC)
print('Congrats! Your evil code may now execute.')

Malicous Javascript:

 b"alert('Ayo, the Wu is back!'); //\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\xe6\x9f\xdf\x80q\xd5\xcd\x07\x87F\xb7@%\xf30\xc6" 

Congrats! Your evil code may now execute.


---

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)